In [5]:
%%javascript
require.config({ 
     paths: { 
     d3: 'https://d3js.org/d3.v4.min'
}});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [8]:
import msprime
import random
import visualizer


# Generate a random tree sequence with record_full_arg=True so that you get marked recombination nodes
rs = random.randint(0,10000)   
ts = msprime.sim_ancestry(
    samples=2,
    recombination_rate=1e-8,
    sequence_length=3_000,
    population_size=10_000,
    record_full_arg=True,
    random_seed=rs
)

#print(ts.tables)

print("random seed:", rs)
print(ts.draw_text())
visualizer = visualizer.D3ARG(ts=ts)
visualizer.draw(width=500, height=500)

random seed: 2181
78143.42┊   18     ┊          ┊         ┊         ┊         ┊  
        ┊ ┏━━┻━━┓  ┊          ┊         ┊         ┊         ┊  
56410.76┊ ┃    17  ┊   17     ┊   17    ┊   17    ┊   17    ┊  
        ┊ ┃     ┃  ┊ ┏━━┻━━┓  ┊  ┏━┻━┓  ┊  ┏━┻━┓  ┊  ┏━┻━┓  ┊  
52603.62┊ 15    ┃  ┊ 16    ┃  ┊ 16   ┃  ┊ 16   ┃  ┊ 16   ┃  ┊  
        ┊ ┃     ┃  ┊ ┃     ┃  ┊  ┃   ┃  ┊  ┃   ┃  ┊  ┃   ┃  ┊  
46579.77┊ ┃    14  ┊ ┃    14  ┊  ┃  14  ┊  ┃  14  ┊  ┃  14  ┊  
        ┊ ┃     ┃  ┊ ┃     ┃  ┊  ┃   ┃  ┊  ┃   ┃  ┊  ┃   ┃  ┊  
44238.59┊ ┃    12  ┊ ┃    12  ┊  ┃  12  ┊  ┃  12  ┊  ┃  13  ┊  
        ┊ ┃     ┃  ┊ ┃     ┃  ┊  ┃   ┃  ┊  ┃   ┃  ┊  ┃   ┃  ┊  
11785.35┊ 11    ┃  ┊ 11    ┃  ┊ 11   ┃  ┊ 11   ┃  ┊ 11   ┃  ┊  
        ┊ ┃     ┃  ┊ ┃     ┃  ┊ ┏┻┓  ┃  ┊ ┏┻┓  ┃  ┊ ┏┻┓  ┃  ┊  
8205.47 ┊ ┃    10  ┊ ┃    10  ┊ ┃ ┃ 10  ┊ ┃ ┃ 10  ┊ ┃ ┃ 10  ┊  
        ┊ ┃   ┏━┻┓ ┊ ┃   ┏━┻┓ ┊ ┃ ┃ ┏┻┓ ┊ ┃ ┃ ┏┻┓ ┊ ┃ ┃ ┏┻┓ ┊  
3387.14 ┊ ┃   9  ┃ ┊ ┃   9  ┃ ┊ ┃ ┃ 9 ┃ ┊ ┃ ┃ 9 ┃ ┊ ┃ ┃ 9 ┃ ┊  
        ┊ ┃  ┏┻┓ ┃ ┊ ┃